<a href="https://colab.research.google.com/github/tomonari-masada/course2023-stats1/blob/main/07_Bayesian_text_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 多項分布を使ったテキスト検索
* 以下の二つの方法を試す。

(1) MAP推定によって推定したパラメータを利用してクエリの確率を求める。

(2) 予測分布を利用してクエリの確率を求める。

**ランタイムのタイプをGPUにしておく**

## (0) 準備

### インポート

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

### データセット

* 20 newsgroupデータセットを使う。
  * 最も類似しているテキストが同じクラスに属するかで検索性能を評価する。
  * （本当は、情報検索の評価専用のデータセットを使う方が良い。）

In [2]:
train_corpus, train_labels = fetch_20newsgroups(subset="train", return_X_y=True)
test_corpus, test_labels = fetch_20newsgroups(subset="test", return_X_y=True)
print(f"training size: {len(train_corpus)}\ntest size: {len(test_corpus)}")

training size: 11314
test size: 7532


### 単語の出現回数を数える

In [3]:
vectorizer = CountVectorizer(min_df=10, stop_words="english")
X_train = vectorizer.fit_transform(train_corpus).toarray()
X_test = vectorizer.transform(test_corpus).toarray()
vocabulary = vectorizer.get_feature_names_out()
print(f"vocabulary size: {len(vocabulary)}")

vocabulary size: 15291


## (1) MAP推定
* 情報検索の世界では「スムージング」と呼ばれる手法に対応する。

### クエリの確率
* クエリの確率を、各文書について求めた単語確率を使って計算する。
  * $c_{\mathbf{x}_0,w}$: クエリ$\mathbf{x}_0$における単語$w$の出現頻度
  * このとき、検索対象の文書$\mathbf{x}$の単語確率$\boldsymbol{\phi}_{\mathbf{x}}$を使って、クエリ$\mathbf{x}_0$の対数確率を表すと、以下のようになる。
$$\begin{align}
\log p(\mathbf{x}_0 ; \boldsymbol{\phi}_{\mathbf{x}}) = \sum_w c_{\mathbf{x}_0,w} \log \phi_{\mathbf{x},w}
\end{align} + const.$$
  * 上の式で、$\mathbf{x}$に依存しない定数は省略して$const.$と書いている。（検索対象のテキストのランキングに関係しないため。）
* このように計算されたクエリの確率の降順に、検索対象のテキストをソートする。
  * $\log p(\mathbf{x}_0 ; \boldsymbol{\phi}_{\mathbf{x}})$が大きい順に、テキスト$\mathbf{x} \in \{ \mathbf{x}_1, \ldots, \mathbf{x}_D\}$を検索結果として表示する。


### 計算の工夫
* $\log p(\mathbf{x}_0 ; \boldsymbol{\phi}_{\mathbf{x}})$の計算は、単に、内積の計算をしているだけ。
  * クエリの単語の出現回数$\mathbf{c}_{\mathbf{x}_0} \equiv (c_{\mathbf{x}_0,1}, \ldots, c_{\mathbf{x}_0,W})$と・・・
  * 検索対象のテキストの単語確率の対数$\log \boldsymbol{\phi}_{\mathbf{x}} \equiv (\log \phi_{\mathbf{x},1}, \ldots, \log\phi_{\mathbf{x},W})$とで・・・
  * 内積の計算$\mathbf{c}_{\mathbf{x}_0}^\top \log \boldsymbol{\phi}_{\mathbf{x}}$をしているだけ。
* ということは・・・
* たくさんあるクエリについて、検索対象のテキストの各々で推定された単語確率で尤度を求めることは、行列の積として書ける。
* 今回は、GPUも使って、高速化する。

* 下のMAP推定では、ディリクレ事前分布のパラメータ$\beta_w$がすべて$\beta$に等しいとする。
  * つまり、対称なディリクレ分布とする。

$$
\begin{align}
\hat{\phi}_{\mathbf{x}, w}
& = \frac{c_{\mathbf{x},w} + \beta_w - 1}{\sum_w (c_{\mathbf{x},w} + \beta_w - 1)}
\notag \\
& = \frac{c_{\mathbf{x},w} + \beta - 1}{l_{\mathbf{x}} + W\beta - W}
\mbox{ （今回の設定ではこうなる。） }
\end{align}
$$

In [6]:
# ディリクレ事前分布のパラメータ
beta = 0.01 + 1.0

X_train = X_train + beta - 1.0
X_train_probs = X_train / X_train.sum(axis=1).reshape(-1, 1)

### クエリの対数尤度を計算するヘルパ関数
* PyTorchを使って、GPU上で計算する。

In [7]:
import torch

def log_likelihood(x_test, x_train_prob):
  return torch.matmul(
      torch.tensor(x_test, dtype=torch.float32, device="cuda"),
      torch.log(torch.tensor(x_train_prob, dtype=torch.float32, device="cuda")).t()
  ).cpu().numpy()

### 検索の実行

* 全てのクエリ（testテキスト）について、その確率を最大にする検索対象のテキスト（trainingテキスト）求める。
  * もちろん、実際の情報検索では、一つ一つのクエリに個別に対応する。
  * 下では、時間節約のため、まとめて計算している。

In [8]:
scores = log_likelihood(X_test, X_train_probs)

In [9]:
sorted_train_indices = (- scores).argsort(-1)

In [10]:
top_ranked_train_docs = sorted_train_indices[:,0].reshape(-1)
print(top_ranked_train_docs)

[9048 4114 7539 ... 2018 2965 7340]


* P@1はprecision at oneの略。
  * https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Precision_at_k

In [11]:
print(f"P@1={(test_labels == train_labels[top_ranked_train_docs]).sum()/len(test_labels):.3f}")

P@1=0.703


* 最上位にランキングされた訓練文書がテスト文書と同じカテゴリになっている割合は、0.7ぐらい。

In [24]:
top_k = 5
precision = (
    train_labels[sorted_train_indices[:,:top_k]]
    == test_labels.reshape(-1, 1)
    ).sum() / (len(test_labels) * top_k)
print(f"P@{top_k}={precision:.3f}")

P@5=0.601


* あとは`beta`をチューニングして性能を出す。

## (2) ベイズ推測
* 予測分布（＝ディリクレ多項分布）を利用してクエリの予測確率を求める。

### クエリの予測確率
* 下の式の$\mathbf{x}$のところに検索対象のテキストを代入して、クエリ$\mathbf{x}_0$の予測確率を計算する。
  * $c_{\mathbf{x},w}$は検索対象のテキストにおける単語$w$の出現回数を表す。
  * $c_{\mathbf{x}_0,w}$はクエリにおける単語$w$の出現回数を表す。

$$
p(\mathbf{x}_0|\mathbf{x};\mathbf{\beta})
= \frac{n_0! \Gamma(\sum_{w=1}^W (c_{\mathbf{x},w} + \beta_w))}{\Gamma( \sum_{w=1}^W (c_{\mathbf{x},w} + c_{\mathbf{x}_0,w} + \beta_w) )}
\prod_{w=1}^W
\frac{\Gamma(c_{\mathbf{x},w} + c_{\mathbf{x}_0,w}+\beta_w)}{c_{\mathbf{x}_0,w}!\Gamma(c_{\mathbf{x},w} + \beta_w)}
$$

* 上の式で、$\mathbf{x}_0$がクエリに相当する。
  * よって、$\mathbf{x}_0$だけに依存する項は、検索対象のテキストのランク付けには無関係。
* ディリクレ事前分布は対称ディリクレ分布だと仮定する。
  * つまり、すべての$w$について$\beta_w = \beta$と、同じ値$\beta$を取ると仮定する。

* 以上を踏まえて、テキスト$\mathbf{x}$を使って算出されるクエリ$\mathbf{x}_0$の対数予測確率を書き下す。
  * テキスト$\mathbf{x}$の長さを$l_{\mathbf{x}}$と書くことにする。

$$
\ln p(\mathbf{x}_0|\mathbf{x}_i;\mathbf{\beta})
= \ln \Gamma(l_\mathbf{x} + W\beta) - \ln \Gamma( l_\mathbf{x} + l_{\mathbf{x}_0} + W \beta )
+ \sum_{w=1}^W \big(
\ln \Gamma(c_{\mathbf{x}, w}+c_{\mathbf{x}_0,w}+\beta_w) - \ln \Gamma(c_{\mathbf{x}, w} + \beta_w) \big) + const.
$$

In [4]:
import torch

X_train_cuda = torch.tensor(X_train, dtype=torch.float32, device="cuda")
train_len = X_train_cuda.sum(-1)
X_test_cuda = torch.tensor(X_test, dtype=torch.float32, device="cuda")
test_len = X_test_cuda.sum(-1)

* 定数の設定

In [5]:
beta = 0.01 #対称ディリクレ事前分布のパラメータ
vocab_size = X_train.shape[-1]

* $\ln \Gamma(l_\mathbf{x} + W\beta)$を計算する。

In [6]:
train_lgamma_all = torch.lgamma(train_len + X_train.shape[-1] * beta)

* $l_\mathbf{x} + l_{\mathbf{x}_0}$をブロードキャストで計算する。
  * 検索対象のテキストとテスト用テキスト（クエリとして使用）の
  * すべての組み合わせについて
  * 二つのテキストの長さの和を求める。

In [7]:
test_train_len = train_len + test_len.unsqueeze(1)

* $\ln \Gamma( l_\mathbf{x} + l_{\mathbf{x}_0} + W \beta )$を計算する。

In [8]:
lgamma_len = torch.lgamma(test_train_len + vocab_size * beta)

In [9]:
lgamma_len.shape

torch.Size([7532, 11314])

* $\ln \Gamma(c_{\mathbf{x}, w} + \beta_w)$を計算する。

In [10]:
train_lgamma_word = torch.lgamma(X_train_cuda + beta)

* $\ln \Gamma(l_\mathbf{x} + W\beta)
- \sum_{w=1}^W
\ln \Gamma(c_{\mathbf{x}, w} + \beta_w)$を計算する。

In [11]:
train_lgamma = train_lgamma_all - train_lgamma_word.sum(-1)

In [12]:
train_lgamma.shape

torch.Size([11314])

* $c_{\mathbf{x}, w}+c_{\mathbf{x}_0,w}$をブロードキャストで計算する。

In [ ]:
#X_sum = X_train_cuda + X_test_cuda.unsqueeze(1)

* メモリが足りないというエラーが出るので、ミニバッチ方式で計算することにする。
  * もちろん、実際の情報検索では、一つ一つのクエリに個別に対応する。

In [13]:
import torch

def log_pred_prob(idx1, idx2):
  X_sum = X_train_cuda + X_test_cuda[idx1:idx2].unsqueeze(1) + beta
  log_prob = train_lgamma.reshape(1, -1) - lgamma_len[idx1:idx2]
  log_prob = log_prob + torch.lgamma(X_sum).sum(-1)
  return log_prob

In [14]:
from tqdm import tqdm

BATCH_SIZE = 4
n_correct_answers = 0
for idx in tqdm(range(0, X_test.shape[0], BATCH_SIZE)):
  sorted_train_indices = (- log_pred_prob(idx, idx+BATCH_SIZE)).argsort(-1)
  top_ranked_train_docs = sorted_train_indices[:,0].reshape(-1)
  n_correct_answers += (
      test_labels[idx:idx+BATCH_SIZE] == train_labels[top_ranked_train_docs.cpu()]
      ).sum()

100%|██████████| 1883/1883 [02:39<00:00, 11.80it/s]


In [ ]:
print(f"P@1={n_correct_answers / len(test_labels):.3f}")

In [16]:
from tqdm import tqdm

BATCH_SIZE = 4
top_k = 5

n_correct_answers = 0
for idx in tqdm(range(0, X_test.shape[0], BATCH_SIZE)):
  sorted_train_indices = (- log_pred_prob(idx, idx+BATCH_SIZE)).argsort(-1)
  n_correct_answers += (
      train_labels[sorted_train_indices[:,:top_k].cpu()]
      == test_labels[idx:idx+BATCH_SIZE].reshape(-1, 1)
      ).sum()
print(f"P@{top_k}={n_correct_answers / (len(test_labels) * top_k):.3f}")

100%|██████████| 1883/1883 [02:37<00:00, 11.94it/s]

P@5=0.580


* あとは`beta`をチューニングする。